<a href="https://colab.research.google.com/github/ciaoCiao100/proveChallenge2/blob/master/customerTensorflowProva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

vengono importate le librerie , scaricato il dataset e impostata la target label

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# operazione che fa in modo che i numeri siano più leggibili
np.set_printoptions(precision=3, suppress=True)

data_path ="/content/drive/My Drive/WA_Fn-UseC_-Telco-Customer-Churn.train.csv"
df = pd.read_csv(data_path)
target_label ="Churn"
df.head()


,Churn,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,No,4223-BKEOR,Female,0,No,Yes,21,Yes,No,DSL,Yes,No,Yes,No,No,Yes,One year,No,Mailed check,64.85,1336.8
1,No,6035-RIIOM,Female,0,No,No,54,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),97.20,5129.45
2,Yes,3797-VTIDR,Male,0,Yes,No,1,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,23.45,23.45
3,Yes,2568-BRGYX,Male,0,No,No,4,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.20,237.95
4,No,2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,NaN


conversione degli elementi presenti in tabella in interi 

In [5]:
#tutte le colonne che contengono stringhe devono essere trasformate in colonne che contengono interi 
for elemento in df:
 
  df[elemento] = pd.Categorical(df[elemento])
  df[elemento] = getattr(df, elemento).cat.codes
# mostro il dataset dopo l'operazione
df.head()


,Churn,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,0,2370,0,0,0,1,21,1,0,0,2,0,2,0,0,2,1,0,3,586,491
1,0,3457,0,0,0,0,54,1,2,1,0,2,0,0,2,2,2,1,0,1170,3510
2,1,2149,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,2,59,1565
3,1,1426,1,0,0,0,4,1,0,1,0,0,0,0,0,0,0,1,2,674,1626
4,0,1545,1,0,0,1,0,1,2,0,2,2,0,2,0,0,2,1,0,546,-1


preparazione del modello della rete

In [60]:
#costruisco il modello
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

model = get_compiled_model()
#preparo il dataset
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

addestramento

In [61]:
# Creazione  del training
train_target = train.pop(target_label)
train_dataset = (tf.data.Dataset.from_tensor_slices((train.values, train_target.values))).shuffle(len(train)).batch(1)
#addestramento e impostazione epoche
model.fit(train_dataset, epochs=500)

Epoch 1/500
3605/3605 [==============================] - 4s 1ms/step - loss: 30.9005 - accuracy: 0.6388
Epoch 2/500
3605/3605 [==============================] - 4s 1ms/step - loss: 6.6709 - accuracy: 0.6863
Epoch 3/500
3605/3605 [==============================] - 4s 1ms/step - loss: 4.4709 - accuracy: 0.6968
Epoch 4/500
3605/3605 [==============================] - 4s 1ms/step - loss: 3.6823 - accuracy: 0.6985
Epoch 5/500
3605/3605 [==============================] - 5s 1ms/step - loss: 2.8128 - accuracy: 0.7126
Epoch 6/500
3605/3605 [==============================] - 4s 1ms/step - loss: 1.8583 - accuracy: 0.7101
Epoch 7/500
3605/3605 [==============================] - 4s 1ms/step - loss: 1.1728 - accuracy: 0.7315
Epoch 8/500
3605/3605 [==============================] - 4s 1ms/step - loss: 0.8714 - accuracy: 0.7304
Epoch 9/500
3605/3605 [==============================] - 5s 1ms/step - loss: 0.6054 - accuracy: 0.7520
Epoch 10/500
3605/3605 [==============================] - 4s 1ms/step - 

test

In [62]:
# Creazione del  test 
test_target = test.pop(target_label)
test_dataset = (tf.data.Dataset.from_tensor_slices((test.values, test_target.values))).shuffle(len(test)).batch(1)

# Fase di test
test_loss, test_accuracy = model.evaluate(test_dataset)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

1127/1127 [==============================] - 1s 920us/step - loss: 0.4481 - accuracy: 0.7622


Test Loss 0.44805920124053955, Test Accuracy 0.7622005343437195


validazione

In [56]:
#valutazione
val_target = val.pop(target_label)

val_dataset = (tf.data.Dataset.from_tensor_slices(val.values)).batch(1)

# Fase di valutazione
predictions = model.predict(val_dataset)


# Show some results 
for prediction, yes in zip(predictions[:10], list(val_target)[:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted yes: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("yes" if bool(va_via) else "no"))


Predicted yes: 39.74%  | Actual outcome:  no
Predicted yes: 16.39%  | Actual outcome:  no
Predicted yes: 1.84%  | Actual outcome:  no
Predicted yes: 3.40%  | Actual outcome:  no
Predicted yes: 41.85%  | Actual outcome:  no
Predicted yes: 1.27%  | Actual outcome:  no
Predicted yes: 15.21%  | Actual outcome:  no
Predicted yes: 41.42%  | Actual outcome:  no
Predicted yes: 0.01%  | Actual outcome:  no
Predicted yes: 62.15%  | Actual outcome:  no
